In [ ]:
from glob import glob
import cv2, os, shutil
import numpy as np
from tqdm import tqdm
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
game_name = '2CTP'
model_name = '02_gulyaz_blue_pony'
project_id = 41
dimensions = [500,200,1520,580]
X, Y, W, H = dimensions

In [ ]:
folder = os.path.join(game_name, model_name)

In [ ]:
if (len(glob(os.path.join(folder, '**', '*.mp4'), recursive=True)) +\
    len(glob(os.path.join(folder, '**', '*.m4v'), recursive=True)) > 0):
    files = glob(os.path.join(folder, '**', '*.mp4'), recursive=True) +\
            glob(os.path.join(folder, '**', '*.m4v'), recursive=True)

In [ ]:
def foo(file):
    game_name, variation_name, *file_name = file.split(os.sep)
    file_name = '_'.join(file_name)
    cap = cv2.VideoCapture(file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cut = file_name.split('_')[0]
    TARGET_DIR = os.path.join('uploads', f'{game_name}_{variation_name}')
    os.makedirs(TARGET_DIR, exist_ok=True)
    if cut not in ['leave']:
        if cut == 'top':
            cap = cv2.VideoCapture(file)
            fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
            out = cv2.VideoWriter(os.path.join(TARGET_DIR, file_name), fourcc, fps, (1920,1080))
            while True:
                ret, frame = cap.read()
                if ret == True:
                    out.write(frame)
                else:
                    break
        else:
            cap = cv2.VideoCapture(file)
            fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
            out = cv2.VideoWriter(os.path.join(TARGET_DIR, file_name), fourcc, fps, (W, H))
            while True:
                ret, frame = cap.read()
                if ret == True:
                    org_shape = frame.shape
                    frame = frame[X:X+H, Y:Y+W]
                    out.write(frame)
                else:
                    break
            

In [ ]:
for file in files:
    print (file)
    foo(file)
    break

In [ ]:
variation_name = game_name + '_' + model_name
folders = glob(os.path.join('uploads', variation_name))
project_ids = {
        variation_name: project_id
}
os.makedirs('excels', exist_ok=True)
for folder in folders:
    videos = sorted(glob(os.path.join(folder, '*.m4v'))) + sorted(glob(os.path.join(folder, '*.mp4')))
    game_name = folder.split(os.sep)[1]
    project_id = project_ids[variation_name]
    l = []
    for idx, video in enumerate(videos):
        file_name = video.split(os.sep)[2]
        print (file_name)
        task_name = file_name.split('.')[0]
        l.append(task_name)
        os.system (f"cvat-cli --server-host http://13.42.37.88 --server-port 8080 \
                    --auth django:django_admin create {task_name} \
                    --project_id {project_id} local {video}")
    pd.DataFrame(l).to_excel(os.path.join('excels', game_name, '.xlsx'), index=False)